In [1]:
#to open cred files
import os

#Data transformation
import pandas as pd

#solve problems with reords
import pandas_gbq

#bigquery
from google.cloud import bigquery

#FIREBASE
import firebase_admin

from firebase_admin import credentials
from firebase_admin import firestore

from dotenv import load_dotenv
load_dotenv('/Users/daniel/OAN/credentials/miong/.env')

print('beginning etl of accounting ... ')

#OPEN CONECTION TO FIREBASE
filename=os.environ['FIREBASE_MIONG_FILENAME']

beginning etl of accounting ... 


In [2]:
## OPEN CONECTION TO BIGQUERRY
bgq_client = bigquery.Client()
dataset_id = "{}.firestore_export".format(bgq_client.project)
dataset = bigquery.Dataset(dataset_id)

In [3]:
cred = credentials.Certificate(filename)
firebase_admin.initialize_app(cred)

db = firestore.client()

In [4]:
#get all the collection of OAN
col_query = db.collection('accountingItems').stream()

In [5]:
#save it in a list of dictionaries
accounts = []
for acc in col_query:
    accounts.append(acc.to_dict())

In [6]:
def convData(acc):
    for item in acc['context']:
        acc[item] = acc['context'][item]
    return acc

In [7]:
accounting = []
for acc in accounts:
    accounting.append(convData(acc))

In [8]:
#convert to dataframe
accounting = pd.DataFrame(accounts)

In [9]:
accounting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4005 entries, 0 to 4004
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   description              3916 non-null   object 
 1   targetIntervention       3991 non-null   object 
 2   vat                      3754 non-null   object 
 3   originProject            3985 non-null   object 
 4   targetAccountingAccount  4005 non-null   object 
 5   originPhase              1719 non-null   object 
 6   context                  4005 non-null   object 
 7   type                     4005 non-null   object 
 8   responsible              4005 non-null   object 
 9   originAccountingAccount  4005 non-null   object 
 10  files                    654 non-null    object 
 11  targetProject            3991 non-null   object 
 12  vatAmount                4005 non-null   float64
 13  amount                   4005 non-null   float64
 14  originIntervention      

In [10]:
df = accounting[['id','account','createdByType','createdBy','lastUpdateBy',
 'createdAt','lastUpdateAt','code','executedAt','responsible',
 'concept','description','type','originProject','originIntervention','originPhase',
 'targetProject','targetIntervention','tags','originAccountingAccount',
 'targetAccountingAccount','amount','baseAmount','vat','vatAmount','files']].copy()

In [24]:
df['files'] = df['files'].apply(lambda x: None if type(x) == type(0.0) else x)

In [25]:
for a in df.columns:
    if a in ['id','account','createdByType','createdBy','lastUpdateBy',
 'createdAt','lastUpdateAt','executedAt','responsible',
 'concept','description','type','originProject','originIntervention','originPhase',
 'targetProject','targetIntervention','originAccountingAccount',
 'targetAccountingAccount']:
    
        df[a] = df[a].astype(str)
    elif a in ['files','tags']:
        df[a] = df[a].apply(lambda x: [str(item) for item in x] if x else [])
    elif a == 'code':
        df[a] = df[a].apply(int)
    else:
        df[a] = df[a].apply(lambda x: float(x) if x 
                                               else 0.0 if x == ''
                                               else 0.0)
        
        

In [26]:
table_id = "{}.accountingItems".format(dataset.dataset_id)
proj = bgq_client.project

In [27]:
job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        # Specify the type of columns whose type cannot be auto-detected. For
        # example the "title" column uses pandas dtype "object", so its
        # data type is ambiguous.
        bigquery.SchemaField("tags", "STRING",mode='REPEATED'),
        # Indexes are written if included in the schema by name.
        bigquery.SchemaField("files", "STRING",mode='REPEATED'),
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

job = bgq_client.load_table_from_dataframe(
    df, table_id, project=bgq_client.project,
    job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

In [15]:
df_oan = df[df['account']=='5Tv2u4n8BReebmKUNIuN'].copy()

In [16]:
df_oanProjet = df_oan.pivot_table('amount','originProject','type',aggfunc='sum').reset_index()

In [17]:
df_oantargProjet = df_oan.pivot_table('amount','targetProject','type',aggfunc='sum').reset_index()

In [18]:
df_oantargProjet['originProject'] = df_oantargProjet['targetProject']
df_oantargProjet['movementOut'] = df_oantargProjet['movement']

In [19]:
df_oanProjetMerge = pd.merge(df_oanProjet,df_oantargProjet[['originProject','movementOut']],
         how='outer',on='originProject')

In [20]:
df_oanProjetMerge['saldo'] = (df_oanProjetMerge['movement'] + df_oanProjetMerge['income'] - 
                         df_oanProjetMerge['expense'] - df_oanProjetMerge['movementOut']
                        )

In [21]:
df_oanProjetMerge

type,originProject,expense,income,movement,movementOut,saldo
0,,NaN,4715.085,NaN,NaN,NaN
1,0DmODGTOEiM5lg9SGx0J,64706.618793,76130.590,82011.60,82011.60,11423.971207
2,1Vf1UJblYQLmUE6lis98,11.480000,NaN,NaN,NaN,NaN
3,4zcptWXv2IqQFkIMz2MP,11254.250000,17732.830,41972.25,41972.25,6478.580000
4,6rV0deyXVSpuu2BfJOGw,1806.440000,NaN,NaN,NaN,NaN
5,95ZS5IjRXGdsbGi2FjJj,4390.540000,4616.000,6572.53,6572.53,225.460000
6,9nACWEXRQYYX1kud5V1s,2154.970000,79.980,5091.46,5091.46,-2074.990000
7,KmRoKdZ3jNI0mbQD56YG,3907.120000,7270.690,2904.30,2904.30,3363.570000
8,RaIH6dfSXt4V7vm6jmVE,497.810000,80.000,NaN,NaN,NaN
9,fzA9DsrefkcRTASwzolq,731.400000,700.930,NaN,NaN,NaN
